In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import simsopt
import simsopt.geo
import simsopt.field
import os

import bdistrib_util
import bdistrib_io


In [ ]:
unpickled_df = pd.read_pickle("QUASR_db/QA_database_26032024.pkl")
unpickled_df.select_dtypes(exclude=["object"]).dtypes


### Test surface extrusion
- Check simple computational boundaries
- Check using a simple scaled up boundary
- Is SVD independent of boundary geometry?

In [ ]:
dfid = 170 #causes problems
lcfs = df["lcfs"][dfid]
curves = [coil.curve for coil in df["coils"][dfid]]

# XYZ tensor fourier -> RZ fourier
psurf = (lcfs.to_RZFourier())
psurf.plot(close=True)

msurf = (lcfs.to_RZFourier())
msurf.scale(1.5)
# msurf.change_resolution(1,1)
msurf.plot(close=True)

osurf = (lcfs.to_RZFourier())
osurf.scale(2)
# osurf.change_resolution(1,1)
osurf.plot(close=True)

# simsopt.geo.plot([osurf, msurf, psurf]) 


In [ ]:
w7xsurf = bdistrib_io.read_netcdf("../bdistrib/equilibria/wout_w7x_standardConfig.nc")
w7xsurf03 = bdistrib_io.read_nescin_file("../bdistrib/equilibria/nescin.w7x_winding_surface_from_Drevlak", 5)
w7xsurf06 = bdistrib_io.read_nescin_file("../bdistrib/equilibria/nescin.w7x_standardConfig_separation0.6", 5)

simsopt.geo.plot([w7xsurf], close=True)
simsopt.geo.plot([w7xsurf, w7xsurf03, w7xsurf06], close=True)


In [ ]:
import scipy.optimize
import functools

def point_distance_in_torus(x, major_r, minor_r):
  # x is ndarray of points with the last axis containing the xyz dimension (,3). All other axes can be shaped arbitrarily
  # Modified from
  # https://math.stackexchange.com/questions/4380905/how-can-i-determine-if-a-point-x-y-z-is-within-a-torus-r-r
  return (major_r - np.sqrt(x[...,0]**2 + x[...,1]**2))**2 + x[...,2]**2 - minor_r**2

def furthest_outside_torus(x, major_r, minor_r):
  return np.max(point_distance_in_torus(x, major_r, minor_r))

def get_enclosing_radius(surface:simsopt.geo.SurfaceRZFourier)->float:
  # Find the minor radius that barely encloses the given fourier surface
  enclosing_minor_radius = scipy.optimize.root_scalar(
    functools.partial(furthest_outside_torus, surface.gamma(), surface.major_radius()), 
    bracket=[surface.minor_radius(), surface.major_radius()])
  if not enclosing_minor_radius.converged:
     raise RuntimeError("Could not find a torus that encloses the given surface")
  return enclosing_minor_radius.root

def make_surf_from_radius(major_r, minor_r, nfp)->simsopt.geo.SurfaceRZFourier:
  wrapping_torus = simsopt.geo.SurfaceRZFourier(5)
  wrapping_torus.set_rc(0,0,major_r)
  wrapping_torus.set_rc(1,0,minor_r)
  wrapping_torus.set_zs(1,0,minor_r)
  return wrapping_torus

# Unit tests for the functions
wrapping_torus = simsopt.geo.SurfaceRZFourier(5)
assert((point_distance_in_torus(wrapping_torus.gamma(), 1, 0.10001)<0).all())
assert(not (point_distance_in_torus(wrapping_torus.gamma(), 1, 0.09999)<0).all())
assert(np.allclose(get_enclosing_radius(wrapping_torus), wrapping_torus.minor_radius()))
# simsopt.geo.plot([psurf, make_surf_from_radius(psurf.major_radius(), get_enclosing_radius(psurf), wrapping_torus.nfp)],engine="plotly")


### Eval using bdistrib

In [ ]:
import subprocess

psurf = w7xsurf
msurf = w7xsurf03
osurf = w7xsurf06

bdistrib_io.write_nescin_file("nescin.msurf", msurf)
bdistrib_io.write_nescin_file("nescin.osurf", osurf)
subprocess.check_call(
    [
        "../bdistrib/bdistrib",
        bdistrib_io.write_bdistribin(
            bdistrib_io.write_netcdf(
                "wout_surfaces_python_generated.nc", psurf
            ),
            geometry_option=3,
            geometry_info={
                "nescin_filename_middle": "nescin.msurf",
                "nescin_filename_outer": "nescin.osurf",
            },
        ),
    ]
)


In [ ]:
import subprocess
import surfgen

# enclosing_r = get_enclosing_radius(psurf)
# outer_offset = 0.2
# assert(enclosing_r+outer_offset < psurf.major_radius())
# subprocess.check_call(["../bdistrib/bdistrib",
#                        write_bdistribin(write_netcdf("wout_surfaces_python_generated.nc", psurf),
#                           geometry_option=1,
#                           geometry_info={
#                             "R0":psurf.major_radius(),
#                             "a_middle":enclosing_r+outer_offset/2,
#                             "a_outer":enclosing_r+outer_offset
#                           })])


def analyze_coil_complexity_at_distance(
    plasma_surface: simsopt.geo.SurfaceRZFourier, offset: float
):
    # msurf = surfgen.surfgen(plasma_surface, offset)
    # assert offset * 2 < plasma_surface.major_radius()
    # osurf = surfgen.surfgen(plasma_surface, offset * 2)

    # bdistrib_io.write_nescin_file("nescin.msurf", msurf)
    # bdistrib_io.write_nescin_file("nescin.osurf", osurf)
    subprocess.check_call(
        [
            "../bdistrib/bdistrib",
            bdistrib_io.write_bdistribin(
                bdistrib_io.write_netcdf(
                    "wout_surfaces_python_generated.nc", plasma_surface
                ),
                geometry_option=3,
                geometry_info={
                    "nescin_filename_middle": "../equilibria/nescin.w7x_winding_surface_from_Drevlak",
                    "nescin_filename_outer": "../equilibria/nescin.w7x_standardConfig_separation0.6",
                },
            ),
        ]
    )
    # Delete Debug Logs cause I don't know how to disable them
    # subprocess.call(["rm", "*.dat"])
    # subprocess.call(["rm", "quasr_coil_check/*.dat"])


analyze_coil_complexity_at_distance(psurf, psurf.minor_radius())

# subprocess.check_call(["../bdistrib/bdistrib",
#                        write_bdistribin(write_netcdf("wout_surfaces_python_generated.nc", psurf),
#                                         geometry_option=2,
#                                         geometry_info={"sep_outer":minimum_coil_surf_distance(curves, psurf)/2}
#                                         )])


In [ ]:
%matplotlib widget
import bdistribPlot
bdistribPlot.main("bdistrib_out.python_generated.nc")


# Something

In [ ]:
for ID in range(1090000, 1150000):
  simsopt_path = bdistrib_io.get_file_path(ID)
  if os.path.exists(simsopt_path):
    soptobj = simsopt.load(simsopt_path)

    lcfs = soptobj[0][-1]
    # XYZ tensor fourier -> RZ fourier
    rzf = lcfs.to_RZFourier()
    curves = [coil.curve for coil in soptobj[1]]
    
    bdistrib_out_path = bdistrib_io.get_file_path(ID, "bdistrib")
    subprocess.check_call(["mkdir", "-p", os.path.dirname(bdistrib_out_path)])
    analyze_coil_complexity_at_distance(rzf, rzf.minor_radius())
    # Move results to correct directory
    subprocess.check_call(["mv", "bdistrib_out.python_generated.nc", bdistrib_out_path, "-u"])
  else:
    print("Skipping", simsopt_path)


In [ ]:
efficiencies = []
for ID in range(2000):
  bdistrib_path = bdistrib_io.get_file_path(ID, "bdistrib")
  if os.path.exists(bdistrib_path):
    avg, stddev = bdistrib_util.rate_of_efficiency_sequence(bdistrib_path)
    efficiencies.append({"efficiency sequence rate of increase":avg,
                        "efficiency sequence rate of increase (dev)":stddev})
df = df.join(pd.DataFrame(efficiencies))


In [ ]:
coil = df["coils"][0][0]
coil.curve


In [ ]:
import plotly.express as px 

px.scatter(df, "volume", "efficiency sequence rate of increase", error_y="efficiency sequence rate of increase (dev)", color="nfp")


In [ ]:
import bdistribPlot

for ID in range(1000, 2000):
  bdistrib_path = bdistrib_io.get_file_path(ID, "bdistrib")
  simsopt_path = bdistrib_io.get_file_path(ID, "simsopt")
  if os.path.exists(bdistrib_path):
    print(bdistrib_path)
    bdistribPlot.main(bdistrib_path)
    # soptobj = simsopt.load(simsopt_path)
    # simsopt.geo.plot(soptobj[0] + soptobj[1], engine="plotly")


### What happens at a sharp corner? (Tokamak)

In [ ]:
tokamak_surf = simsopt.geo.SurfaceRZFourier(nfp=2, stellsym=False, mpol=5, ntor=5)
tokamak_surf.set_zs(1,0, 0.5)
tokamak_surf.set_rc(1,0, 0.4)
tokamak_surf.set_rs(1,0, 0.1)
tokamak_surf.set_rs(2,0, 0.2)

subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(write_netcdf(tokamak_surf), 
                                                                sep_outer=0.2)])

bdistribPlot.main("bdistrib_out.tokamak.nc")


## Eval B.n on computational surfaces using the coils

In [ ]:
def eval_row(row, plot=False):
  coils = row["coils"]
  nfp = row["nfp"]
  R1 = row["R1"]
  lcfs = row["lcfs"]
  computational_surface = simsopt.geo.SurfaceRZFourier.from_nphi_ntheta(nfp=nfp, nphi=64, ntheta=64, range="half period")

  # Simple torus boundary
  computational_surface.set_rc(1,0,R1*2.0)
  computational_surface.set_zs(1,0,R1*2.0)
  computational_surface.change_resolution(3,4)
  # computational_surface.set_rc(2,1,np.random.rand())
  # computational_surface.set_zs(1,2,np.random.rand())
  normal_computational = computational_surface.normal()
  xyz_computational = computational_surface.gamma()

  # Scaled computational boundary
  scale = 1.5
  lcfs.scale(scale)
  xyz_computational = lcfs.gamma()
  normal_computational = lcfs.normal()
  lcfs.scale(1.0 / scale)

  bs = simsopt.field.BiotSavart(coils)
  bs.set_points_cart(xyz_computational.reshape((-1,3)))
  B = bs.B().reshape(xyz_computational.shape)
  BdotN = np.sum(normal_computational * B, axis=-1)
  BdotN_fft = np.fft.fft2(BdotN)
  
  if plot:
    import plotly.express as px
    # px.imshow(BdotN).show()
    # px.imshow(np.abs(np.fft.fftshift(BdotN_fft))).show()
    # px.imshow(np.real(np.fft.fftshift(BdotN_fft)), title="Real component").show()
    # px.imshow(np.imag(np.fft.fftshift(BdotN_fft)), title="Imag component").show()
    plt.figure(figsize=(16,5))
    plt.subplot(131)
    plt.imshow(BdotN)
    plt.title("BdotN")
    plt.colorbar()
    plt.subplot(132)
    plt.imshow(np.real(np.fft.fftshift(BdotN_fft)))
    plt.title("fft real")
    plt.colorbar()
    plt.subplot(133)
    plt.imshow(np.imag(np.fft.fftshift(BdotN_fft)))
    plt.title("fft imag")
    plt.colorbar()
    # plt.tight_layout()
    plt.show()

  return BdotN_fft

for idx,row in df.iterrows():
  eval_row(row, False)
